In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Augmentor

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.applications.mobilenet import MobileNet
import os
from keras.preprocessing.image import ImageDataGenerator
import glob

In [ ]:
ROOT_PATH    = "/content/drive/MyDrive/MalariaDatasetNEW"
NORMAL_PATH  = os.path.join(ROOT_PATH, 'NORMAL')
MALARIA_PATH = os.path.join(ROOT_PATH, 'MALARIA')

In [ ]:
TRAIN_PATH = os.path.join(ROOT_PATH, "Train")
TEST_PATH  = os.path.join(ROOT_PATH, "Test")

In [ ]:
NORMAL = glob.glob(ROOT_PATH+"/NORMAL/*") 
MALARIA = glob.glob(ROOT_PATH+"/MALARIA/*")

len(NORMAL), len(MALARIA)

(35810, 550)

In [ ]:
len(NORMAL)

35810

In [ ]:
normal_train_len = 1250
normal_test_len = 500
normal_tot_len = normal_train_len + normal_test_len

normal_idxs = np.random.choice(len(NORMAL), normal_tot_len, replace=False)

malaria_train_len = 525
malaria_test_len = 25
malaria_tot_len = malaria_train_len + malaria_test_len

malaria_idxs = np.random.choice(len(MALARIA), malaria_tot_len, replace=False)

In [ ]:
len(normal_idxs), len(malaria_idxs)

(1750, 550)

In [ ]:
target_path = os.path.join(TRAIN_PATH, "MALARIA")
target_path = os.path.join(target_path, "TEST")
target_path

'/content/drive/MyDrive/MalariaDatasetNEW/Train/MALARIA/TEST'

In [ ]:
import shutil

malaria_files = os.listdir(MALARIA_PATH)

malaria_train_idxs = malaria_idxs[:malaria_train_len]
malaria_test_idxs  = malaria_idxs[malaria_train_len:]

print(len(malaria_train_idxs), len(malaria_test_idxs))

for i, idx in enumerate(malaria_train_idxs): 
    img_path = os.path.join(MALARIA_PATH, malaria_files[idx])
    target_path = os.path.join(TRAIN_PATH, "MALARIA")
    target_path = os.path.join(target_path, str(i).rjust(4, '0')+".png")
    shutil.copyfile(img_path, target_path)

for i, idx in enumerate(malaria_test_idxs): 
    img_path = os.path.join(MALARIA_PATH, malaria_files[idx])
    target_path = os.path.join(TEST_PATH, "MALARIA")
    target_path = os.path.join(target_path, str(i).rjust(4, '0')+".png")
    shutil.copyfile(img_path, target_path)

525 25


In [ ]:
len(glob.glob(TRAIN_PATH + "/MALARIA/*")), len(glob.glob(TEST_PATH + "/MALARIA/*"))

In [ ]:
normal_files = os.listdir(NORMAL_PATH)

normal_train_idxs = normal_idxs[:normal_train_len]
normal_test_idxs  = normal_idxs[normal_train_len:]

print(len(normal_train_idxs), len(normal_test_idxs))

for i, idx in enumerate(normal_train_idxs): 
    img_path = os.path.join(NORMAL_PATH, normal_files[idx])
    target_path = os.path.join(TRAIN_PATH, "NORMAL")
    target_path = os.path.join(target_path, str(i).rjust(4, '0')+".png")
    shutil.copyfile(img_path, target_path)

for i, idx in enumerate(normal_test_idxs): 
    img_path = os.path.join(NORMAL_PATH, normal_files[idx])
    target_path = os.path.join(TEST_PATH, "NORMAL")
    target_path = os.path.join(target_path, str(i).rjust(4, '0')+".png")
    shutil.copyfile(img_path, target_path)

1250 500


In [ ]:
len(glob.glob(TRAIN_PATH + "/NORMAL/*")), len(glob.glob(TEST_PATH + "/NORMAL/*"))

(1250, 500)

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 8

In [ ]:
len(glob.glob(ROOT_PATH+"/NORMAL/*.*") + glob.glob(ROOT_PATH+"/MALARIA/*.*"))

36360

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    directory = ROOT_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

In [ ]:
from sys import tracebacklimit
train = tf.keras.utils.image_dataset_from_directory(
            directory=ROOT_PATH,
            labels='inferred',
            label_mode='int',
            class_names=['NORMAL', 'MALARIA'],
            color_mode='rgb',
            batch_size=8,
            image_size=(128, 128),
            shuffle=True,
            seed=None,
            validation_split=0.1,
            subset="training",
            interpolation='bilinear',
            follow_links=False,
            crop_to_aspect_ratio=False
        )

validation = train = tf.keras.utils.image_dataset_from_directory(
                        directory=ROOT_PATH,
                        labels='inferred',
                        label_mode='int',
                        class_names=['NORMAL', 'MALARIA'],
                        color_mode='rgb',
                        batch_size=8,
                        image_size=(128, 128),
                        shuffle=True,
                        seed=None,
                        validation_split=0.1,
                        subset="training",
                        interpolation='bilinear',
                        follow_links=False,
                        crop_to_aspect_ratio=False
                        
                    )

Found 544 files belonging to 2 classes.


tensorflow.python.data.ops.dataset_ops.BatchDataset

In [ ]:
ds.as_dataset()

In [ ]:
ds.class_names

['NORMAL', 'MALARIA']

In [ ]:
class_names = ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
model = MobileNet(
    input_shape=(128, 128, 3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)